# Problema da coleta de leite

**Conjuntos:**
- $N$: Conjunto de nós, onde cada nó representa um produtor (incluindo o depósito).
- $N_d$: conjunto de produtores que requerem coleta diária.
- $N_a$: conjunto de produtores que requerem coleta em dias alternados.
- $D$: conjunto dos dois dias de operação (Dia 1 e Dia 2).

**Parâmetros:**
- $d_{ij}$: Distância euclidiana entre os nós $i$ e $j$.
- $q_i$: Demanda diária do produtor $i$.
- $Q$: capacidade máxima do caminhão em litros.

**Variáveis de Decisão:**
- $x_{dij}$: Variável binária que indica se a rota $(i,j)$ é usada no dia $d$.
- $u_{di}$: Variável contínua que representa a posição de um nó na rota no dia $d$.
- $v_{di}$: Variável contínua que indica a quantidade coletada do produtor $i$ no dia $d$.

**Função Objetivo:**
$$
\text{Minimize} \quad \sum_{d \in D} \sum_{(i,j) \in N \times N} d_{ij} \cdot x_{dij}
$$

**Restrições:**
- **Grau diário de entrada:** Para cada produtor $i$ nos dias de coleta diária:
  $$
  \sum_{j \in N} x_{dji} = 1, \quad \forall d \in D, \forall i \in N_d
  $$
- **Grau diário de saída:** Para cada produtor $i$ nos dias de coleta diária:
  $$
  \sum_{j \in N} x_{dij} = 1, \quad \forall d \in D, \forall i \in N_d
  $$
- **Grau alternativo:** Para cada produtor $i$ nos dias de coleta alternativa:
  $$
  \sum_{d \in D} \sum_{j \in N, k \in N} (x_{dji} + x_{dik}) \geq 2, \quad \forall i \in N_a
  $$
- **Conservação do fluxo:** Para cada dia $d$ e produtor $i$:
  $$
  \sum_{j \in N} x_{dij} = \sum_{j \in N} x_{dji}, \quad \forall d \in D, \forall i \in N
  $$
- **Eliminação de subtours:** Para cada par de nós $i$ e $j$ diferente do depósito:
  $$
  u_{dj} \geq u_{di} + (|N|-1) \cdot x_{dij} + (|N|-3) \cdot x_{dji} - (|N|-2), \quad \forall d \in D, \forall (i,j) \in N \times N
  $$
- **Capacidade:** A capacidade total de coleta não deve exceder $Q$:
  $$
  \sum_{i \in N} v_{di} \leq Q, \quad \forall d \in D
  $$
- **Demanda diária:** Para cada produtor $i$ nos dias de coleta diária:
  $$
  v_{di} = q_i, \quad \forall d \in D, \forall i \in N_d
  $$
- **Demanda alternativa:** Para cada produtor $i$ nos dias de coleta alternativa:
  $$
  \sum_{d \in D} v_{di} = q_i, \quad \forall i \in N_a
  $$
- **Ativação da coleta:** Garante que a coleta só seja ativada se alguma rota chegar ou sair do produtor $i$:
  $$
  v_{di} \leq q_i \cdot \left( \sum_{j \in N, j \neq i} x_{dji} + \sum_{k \in N, k \neq i} x_{dik} \right), \quad \forall d \in D, \forall i \in N
  $$

In [ ]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from math import sqrt
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

data = {
    'Produtor': range(1, 22),
    'Leste': [0, -3, 1, 4, -5, -5, -4, 6, 3, -1, 0, 6, 2, -2, 6, 1, -3, -6, 2, -6, 5],
    'Norte': [0, 3, 11, 7, 9, -2, -7, 0, -6, -3, -6, 4, 5, 8, 10, 8, 1, 5, 9, -5, -4]
}

demand = {
    0: 0 , 1: 5000, 2: 4000, 3: 3000, 4: 6000, 5: 7000, 6: 3000, 7: 4000, 8: 6000, 9: 5000,
    10: 4000, 11: 7000, 12: 3000, 13: 4000, 14: 5000, 15: 6000, 16: 8000, 17: 5000, 18: 7000, 19: 6000, 20: 6000
}

Q = 80000
daily = {1, 2, 3, 4, 5, 6, 7, 8, 9}
alternate = {10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
locations = pd.DataFrame(data)
locations['Leste'] = locations['Leste'] * 10
locations['Norte'] = locations['Norte'] * 10

# Distância Euclidiana
def calc_distance(row1, row2):
    return sqrt((row1['Leste'] - row2['Leste'])**2 + (row1['Norte'] - row2['Norte'])**2)

# Matriz de distância
distance_matrix = np.zeros((len(locations), len(locations)))
for i in range(len(locations)):
    for j in range(len(locations)):
        distance_matrix[i][j] = calc_distance(locations.iloc[i], locations.iloc[j])

# Lista de vertices (produtores incluindo o depósito)
N = list(range(len(locations)))
D = [1, 2]


In [16]:
def daily_degree_in(model, d, i):
    if i in daily:
        return sum(model.x[d, j, i] for j in model.N if (j, i) in model.Pairs) == 1
    return pyo.Constraint.Skip

def daily_degree_out(model, d, i):
    if i in daily:
        return sum(model.x[d, i, j] for j in model.N if (i, j) in model.Pairs) == 1
    return pyo.Constraint.Skip

def alternate_degree(model, i):
    if i in alternate:        
        return sum(model.x[d, j, i] + model.x[d, i, k] for d in model.D for j in model.N for k in model.N if j != i and k != i) >= 2        
    else:
        return pyo.Constraint.Skip

def flow_rule(model, d, i):    
    return sum(model.x[d, i, j] for j in model.N if (i, j) in model.Pairs) == sum(model.x[d, j, i] for j in model.N if (j, i) in model.Pairs)    

def subtour_elimination(model, d, i, j):        
    if i != 0 and j != 0 :        
        return model.u[d,j] >= model.u[d,i] + (len(N)-1)*model.x[d,i,j] + (len(N)-3)*model.x[d,j,i] -  (len(N)-2)
    else:
        return pyo.Constraint.Skip

def capacity_constraint(model, d):
    return sum(model.v[d, i] for i in model.N) <= Q

def daily_demand(model, d, i):  
    if i in daily:      
        return model.v[d, i] == model.q[i] 
    else:
        return pyo.Constraint.Skip

def alternate_demand(model, i):
    if i in alternate:        
        return sum(model.v[d, i] for d in model.D) == model.q[i]
    else:
        return pyo.Constraint.Skip
    
def collection_activation(model, d, i):
    if i != 0:        
        return model.v[d, i] <= model.q[i] * (sum(model.x[d, j, i] for j in model.N if j != i) + sum(model.x[d, i, k] for k in model.N if k != i))
    else:
        return pyo.Constraint.Skip


def create_coleta_model(N, distance_matrix):
    model = pyo.ConcreteModel()
    
    # Conjuntos
    model.N = pyo.Set(initialize=N)
    model.Pairs = pyo.Set(within=model.N*model.N, initialize=[(i,j) for i in N for j in N if i!=j])
    model.D = pyo.Set(initialize=D)
      
    # Parâmetros
    model.d = pyo.Param(model.Pairs, initialize={(i,j): distance_matrix[i][j] for i,j in model.Pairs})
    model.q = pyo.Param(model.N, initialize={(i): demand[i] for i in model.N})    
    
    # Variáveis de decisão
    model.x = pyo.Var(model.D, model.Pairs, domain=pyo.Binary)
    model.u = pyo.Var(model.D, model.N, within=pyo.NonNegativeReals)
    model.v = pyo.Var(model.D, model.N, domain=pyo.NonNegativeReals)
    
    # Função objetivo
    model.obj = pyo.Objective(expr=sum(model.d[i,j] * model.x[d,i,j] for d in model.D for i,j in model.Pairs), sense=pyo.minimize)    
    
    # Restrições        
    model.daily_degree_in = pyo.Constraint(model.D, model.N, rule=daily_degree_in)
    model.daily_degree_out = pyo.Constraint(model.D, model.N, rule=daily_degree_out)
    model.alternate_degree = pyo.Constraint(model.N, rule=alternate_degree)
    model.flow = pyo.Constraint(model.D, model.N, rule=flow_rule)
    model.subtour_elimination = pyo.Constraint(model.D, model.Pairs, rule=subtour_elimination)
    for d in model.D:
        model.u[d,0].fix(0)
    model.capacity = pyo.Constraint(model.D, rule=capacity_constraint)
    model.daily_demand = pyo.Constraint(model.D, model.N, rule=daily_demand)  
    model.alternate_demand = pyo.Constraint(model.N, rule=alternate_demand)   
    model.collection_activation = pyo.Constraint(model.D, model.N, rule=collection_activation)
    
    return model

In [17]:
def solve(model):
    solver = SolverFactory('gurobi')    
    solver.options['TimeLimit'] = 10
    result = solver.solve(model, tee=True)    
    
    if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
        print("Solution is optimal")
    elif result.solver.termination_condition == TerminationCondition.infeasible:
        print("No feasible solution found")
    else:
        print("Solver Status: ", result.solver.status)
        
    print("Objective Value: ", model.obj())
    print("Gurobi Solver Time: ", result.solver.time, "seconds")

model_coleta = create_coleta_model(N, distance_matrix)
solve(model_coleta)

Set parameter CSManager to value "http://110.22.10.6:61080"
Set parameter CSAuthToken
Compute Server job ID: 571232d6-d2cc-4549-821a-336908ccf0fe
Capacity available on 'SRVAIOPT01' - connecting...
Established HTTP unencrypted connection
Read LP format model from file C:\Users\JEANLU~1.MOT\AppData\Local\Temp\tmpmf945n7o.pyomo.lp
Reading time = 0.68 seconds
x1: 920 rows, 922 columns, 8042 nonzeros
Set parameter TimeLimit to value 70
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Gurobi Compute Server Worker version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 920 rows, 922 columns and 8042 nonzeros
Model fingerprint: 0xf86ecb01
Variable types: 82 continuous, 840 integer (840 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [1e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+04]
Found heuristic solution: objective 5218.8193250
Presolve

## Visualização da solução

In [38]:
def print_routes(model):
    for d in model.D:
        print(f"Rotas para o Dia {d}:")
        visited = set()  # Para controlar os nós já visitados
        i = 0
        if i not in visited:
            current = i
            route = f"{i + 1}"  # Ajusta a visualização para base 1
            visited.add(i)
            while True:
                found = False
                for j in model.N:
                    if (current, j) in model.Pairs and model.x[d, current, j].value > 0.5:
                        if j not in visited:
                            route += f" -> {j + 1}"  # Ajusta para base 1
                            visited.add(j)
                            current = j
                            found = True
                            break
                if not found or current == 0:  # Se não encontrou nenhum ou voltou ao depósito
                    break
            print(route + " -> 1")

def print_idle_capacity(model):    
    for d in model.D:
        total_usage = 0 
        for i in model.N:     
            total_usage += model.v[d,i].value       
        print(f"Capacidade ociosa do dia {d}: {Q-total_usage:.2f} litros")        

print_routes(model_coleta)
print("Objective Value: ", round(model_coleta.obj(),2))
print_idle_capacity(model_coleta)

Rotas para o Dia 1:
1 -> 2 -> 5 -> 3 -> 19 -> 16 -> 4 -> 12 -> 8 -> 9 -> 11 -> 7 -> 6 -> 10 -> 1
Rotas para o Dia 2:
1 -> 17 -> 2 -> 18 -> 5 -> 14 -> 3 -> 15 -> 4 -> 13 -> 8 -> 21 -> 9 -> 7 -> 20 -> 6 -> 10 -> 1
Objective Value:  1230.56
Capacidade ociosa do dia 1: 12999.99 litros
Capacidade ociosa do dia 2: 0.01 litros


**A rota ótima nos dois dias é:**

- **Dia 1:** 1 -> 2 -> 5 -> 3 -> 19 -> 16 -> 4 -> 12 -> 8 -> 9 -> 11 -> 7 -> 6 -> 10 -> 1
- **Dia 2:** 1 -> 17 -> 2 -> 18 -> 5 -> 14 -> 3 -> 15 -> 4 -> 13 -> 8 -> 21 -> 9 -> 7 -> 20 -> 6 -> 10 -> 1

**A distância mínima percorrida é de** 1230.56 km.

**A capacidade ociosa do caminhão de coleta nos dois dias é:**
- **Dia 1:** 12,999.99 litros
- **Dia 2:** 0.01 litros